Load data

In [78]:
import pandas as pd
import glob
import os

path = "scale_data/scaledata/Dennis+Schwartz/"

all_files = glob.glob(path + "*")

def read_csv(f):
    return pd.read_csv(f, engine="python", delimiter="impossible_delimiter_to_read_line_by_line", names=[os.path.basename(f).rsplit(".", 1)[0]])

def reset_index(df):
    df.reset_index(drop=True, inplace=True)

dataframes = [read_csv(f) for f in all_files]

df = pd.concat(dataframes, axis=1)


df.head()


,id,label.3class,label.4class,rating,subj
0,29420,0,0,0.1,"in my opinion , a movie reviewer's most import..."
1,17219,0,0,0.2,"you can watch this movie , that is based on a ..."
2,18406,0,0,0.2,"this is asking a lot to believe , and though i..."
3,18648,0,0,0.2,no heroes and no story are the main attributes...
4,20021,0,0,0.2,"this is not an art movie , yet i saw it an art..."
